In [6]:
from data_generator import DataGenerator
# from embeddings_generator import *


# Hyperparameters

# if __name__ == '__main__':

history_length = 12 # N in article
ra_length = 4 # K in article
discount_factor = 0.99 # Gamma in Bellman equation
actor_lr = 0.0001
critic_lr = 0.001
tau = 0.001 # τ in Algorithm 3
batch_size = 64
nb_episodes = 100
nb_rounds = 50
filename_summary = 'summary.txt'
alpha = 0.5 # α (alpha) in Equation (1)
gamma = 0.9 # Γ (Gamma) in Equation (4)
buffer_size = 1000000 # Size of replay memory D in article
fixed_length = True # Fixed memory length

dg = DataGenerator('ml-100k/u.data', 'ml-100k/u.item')
dg.gen_train_test(0.8, seed=42)

# dg.write_csv('train.csv', dg.train, nb_states=[history_length], nb_actions=[ra_length])
# dg.write_csv('test.csv', dg.test, nb_states=[history_length], nb_actions=[ra_length])


In [22]:
import random
def sample_histo_v1(user_histo, action_ratio=0.8, max_samp_by_user=5,  max_state=100, max_action=50, nb_states=[], nb_actions=[]):
    '''
    For a given historic, make one or multiple sampling.
    If no optional argument given for nb_states and nb_actions, then the sampling
    is random and each sample can have differents size for action and state.
    To normalize sampling we need to give list of the numbers of states and actions
    to be sampled.

    Parameters
    ----------
    user_histo :  DataFrame
                      historic of user
    delimiter :       string, optional
                      delimiter for the csv
    action_ratio :    float, optional
                      ratio form which movies in history will be selected
    max_samp_by_user: int, optional
                      Number max of sample to make by user
    max_state :       int, optional
                      Number max of movies to take for the 'state' column
    max_action :      int, optional
                      Number max of movies to take for the 'action' action
    nb_states :       array(int), optional
                      Numbers of movies to be taken for each sample made on user's historic
    nb_actions :      array(int), optional
                      Numbers of rating to be taken for each sample made on user's historic

    Returns
    -------
    states :         List(String)
                     All the states sampled, format of a sample: itemId&rating
    actions :        List(String)
                     All the actions sampled, format of a sample: itemId&rating


    Notes
    -----
    States must be before(timestamp<) the actions.
    If given, size of nb_states is the numbller of sample by user
    sizes of nb_states and nb_actions must be equals
    '''

    n = len (user_histo)
    sep = int (action_ratio * n)
    nb_sample = random.randint (1, max_samp_by_user)
    if not nb_states:
        nb_states = [min (random.randint (1, sep), max_state) for i in range (nb_sample)]
    if not nb_actions:
        nb_actions = [min (random.randint (1, n - sep), max_action) for i in range (nb_sample)]
    assert len (nb_states) == len (nb_actions), 'Given array must have the same size'

    states = []
    actions = []
    # SELECT SAMPLES IN HISTO
    for i in range (len (nb_states)):
        sample_states = user_histo.iloc[0:sep].sample (nb_states[i])
        sample_actions = user_histo.iloc[-(n - sep):].sample (nb_actions[i])

        sample_state = []
        sample_action = []
        for j in range (nb_states[i]):
            row = sample_states.iloc[j]
            # FORMAT STATE
            state = str (row.loc['itemId']) + '&' + str (row.loc['rating'])
            sample_state.append (state)

        for j in range (nb_actions[i]):
            row = sample_actions.iloc[j]
            # FORMAT ACTION
            action = str (row.loc['itemId']) + '&' + str (row.loc['rating'])
            sample_action.append (action)


        states.append (sample_state)
        actions.append (sample_action)

    return states, actions

In [13]:
import random
def sample_histo_v2(user_histo, action_ratio=0.8, max_samp_by_user=5,  max_state=100, max_action=50, nb_states=[], nb_actions=[]):
    '''
    For a given historic, make one or multiple sampling.
    If no optional argument given for nb_states and nb_actions, then the sampling
    is random and each sample can have differents size for action and state.
    To normalize sampling we need to give list of the numbers of states and actions
    to be sampled.

    Parameters
    ----------
    user_histo :  DataFrame
                      historic of user
    delimiter :       string, optional
                      delimiter for the csv
    action_ratio :    float, optional
                      ratio form which movies in history will be selected
    max_samp_by_user: int, optional
                      Number max of sample to make by user
    max_state :       int, optional
                      Number max of movies to take for the 'state' column
    max_action :      int, optional
                      Number max of movies to take for the 'action' action
    nb_states :       array(int), optional
                      Numbers of movies to be taken for each sample made on user's historic
    nb_actions :      array(int), optional
                      Numbers of rating to be taken for each sample made on user's historic

    Returns
    -------
    states :         List(String)
                     All the states sampled, format of a sample: itemId&rating
    actions :        List(String)
                     All the actions sampled, format of a sample: itemId&rating


    Notes
    -----
    States must be before(timestamp<) the actions.
    If given, size of nb_states is the numbller of sample by user
    sizes of nb_states and nb_actions must be equals
    '''

    n = len(user_histo)
    sep = int (action_ratio * n)
    nb_sample = random.randint (1, max_samp_by_user)
    if not nb_states:
        nb_states = [min (random.randint (1, sep), max_state) for i in range (nb_sample)]
    if not nb_actions:
        nb_actions = [min (random.randint (1, n - sep), max_action) for i in range (nb_sample)]
    assert len (nb_states) == len (nb_actions), 'Given array must have the same size'
    
    states = []
    actions = []
    for i in range (len (nb_states)):
        user_states = user_histo[user_histo['rating']>=4]
        if len(user_states) >= nb_states[i]:
            sample_states = user_states.sample (nb_states[i])
        else:
            continue

        sample_actions = user_histo.iloc[-(n - sep):].sample (nb_actions[i])

        sample_state = []
        sample_action = []

        for j in range (nb_states[i]):
            row = sample_states.iloc[j]
            # FORMAT STATE
            state = str (row.loc['itemId']) + '&' + str (1)
            sample_state.append (state)

        for k in range (nb_actions[i]):
            row = sample_actions.iloc[k]
            # FORMAT ACTION
            if row.loc['rating'] >= 4:
                action = str (row.loc['itemId']) + '&' + str (1)
            else:
                action = str (row.loc['itemId']) + '&' + str (0)
            sample_action.append (action)

        states.append (sample_state)
        actions.append (sample_action)
    return states, actions

In [19]:
import random
import copy
def sample_histo_v3(user_histo, action_ratio=0.8, max_samp_by_user=5,  max_state=100, max_action=50, nb_states=[], nb_actions=[]):
    '''
    For a given historic, make one or multiple sampling.
    If no optional argument given for nb_states and nb_actions, then the sampling
    is random and each sample can have differents size for action and state.
    To normalize sampling we need to give list of the numbers of states and actions
    to be sampled.

    Parameters
    ----------
    user_histo :  DataFrame
                      historic of user
    delimiter :       string, optional
                      delimiter for the csv
    action_ratio :    float, optional
                      ratio form which movies in history will be selected
    max_samp_by_user: int, optional
                      Number max of sample to make by user
    max_state :       int, optional
                      Number max of movies to take for the 'state' column
    max_action :      int, optional
                      Number max of movies to take for the 'action' action
    nb_states :       array(int), optional
                      Numbers of movies to be taken for each sample made on user's historic
    nb_actions :      array(int), optional
                      Numbers of rating to be taken for each sample made on user's historic

    Returns
    -------
    states :         List(String)
                     All the states sampled, format of a sample: itemId&rating
    actions :        List(String)
                     All the actions sampled, format of a sample: itemId&rating


    Notes
    -----
    States must be before(timestamp<) the actions.
    If given, size of nb_states is the numbller of sample by user
    sizes of nb_states and nb_actions must be equals
    '''

    n = len(user_histo)
    sep = int (action_ratio * n)
    nb_sample = random.randint (1, max_samp_by_user)
    if not nb_states:
        nb_states = [min (random.randint (1, sep), max_state) for i in range (nb_sample)]
    if not nb_actions:
        nb_actions = [min (random.randint (1, n - sep), max_action) for i in range (nb_sample)]
    assert len (nb_states) == len (nb_actions), 'Given array must have the same size'
    
    states = []
    actions = []
    rewards = []
    for n in range(len(nb_states)):
        state_len = nb_states[n]
        action_len = nb_actions[n]

        item_list = user_histo['itemId'].values.tolist()
        click_list = user_histo['rating'].values.tolist()
        initial_state = []
        initial_end = 0
        for i in range(len(item_list)):
            if click_list[i] >= 4 and len(initial_state) < state_len:
                initial_state.append(item_list[i])
                initial_end = i



        if len(initial_state) == state_len and (initial_end + action_len <= len(item_list)):
            current_state = copy.copy(initial_state)
            for i in range(initial_end+1,len(item_list),action_len):
                if i+action_len <= len(item_list):

                    actions.append([str(item) + '&' + str (1) if rating >= 4 else str(item) + '&' + str (0) for item,rating in zip(item_list[i:i+action_len],click_list[i:i+action_len])])
                    rewards.append(click_list[i:i+action_len])
                    states.append([str(i) + '&' + str (1) for i in copy.copy(current_state)])
                    for j in range(i,i+action_len):
                        if click_list[j] == 1:
                            current_state.append(item_list[j])
                            del current_state[0]
    return states, actions

In [59]:
user_histo = dg.train[0]

In [8]:
user_histo = dg.train[0]
action_ratio=0.8
max_samp_by_user=5
max_state=100
max_action=50
nb_states=[12]
nb_actions=[4]

In [9]:
n = len (user_histo)
sep = int (action_ratio * n)
nb_sample = random.randint (1, max_samp_by_user)
n,sep,nb_sample

(271, 216, 2)

In [10]:
if not nb_states:
    nb_states = [min (random.randint (1, sep), max_state) for i in range (nb_sample)]
if not nb_actions:
    nb_actions = [min (random.randint (1, n - sep), max_action) for i in range (nb_sample)]
assert len (nb_states) == len (nb_actions), 'Given array must have the same size'

In [24]:
for user_histo in dg.train:
    states, actions = sample_histo_v1(user_histo,nb_states=[12],nb_actions=[4])
#     print(states)
#     print(actions)
#     break
    for i in range (len (states)):
        # FORMAT STATE
        state_str = '|'.join (states[i])
        print(state_str)
        # FORMAT ACTION
        action_str = '|'.join (actions[i])
        print(action_str)
        # FORMAT N_STATE
        n_state_str = state_str + '|' + action_str
        print(n_state_str)
        break
    break

197&5|428&3|168&5|486&4|181&4|202&5|238&5|477&4|234&4|480&5|367&4|240&3
722&2|1052&2|502&4|525&4
197&5|428&3|168&5|486&4|181&4|202&5|238&5|477&4|234&4|480&5|367&4|240&3|722&2|1052&2|502&4|525&4


In [25]:
for user_histo in dg.train:
    states, actions = sample_histo_v2(user_histo,nb_states=[12],nb_actions=[4])
#     print(states)
#     print(actions)
#     break
    for i in range (len (states)):
        # FORMAT STATE
        state_str = '|'.join (states[i])
        print(state_str)
        # FORMAT ACTION
        action_str = '|'.join (actions[i])
        print(action_str)
        # FORMAT N_STATE
        n_state_str = state_str + '|' + action_str
        print(n_state_str)
        break
    break

8&1|613&1|205&1|435&1|945&1|836&1|42&1|492&1|478&1|367&1|98&1|209&1
579&0|715&0|722&0|88&0
8&1|613&1|205&1|435&1|945&1|836&1|42&1|492&1|478&1|367&1|98&1|209&1|579&0|715&0|722&0|88&0


In [31]:
for user_histo in dg.train:
    states, actions = sample_histo_v3(user_histo,nb_states=[12],nb_actions=[4])
    print(len(states))
#     print(actions)
#     break
    for i in range (len (states)):
        # FORMAT STATE
        state_str = '|'.join (states[i])
#         print(state_str)
        # FORMAT ACTION
        action_str = '|'.join (actions[i])
#         print(action_str)
        # FORMAT N_STATE
        n_state_str = state_str + '|' + action_str
#         print(n_state_str)
        
        

63


53


2


12


50


2


7


39


KeyboardInterrupt: Interrupted by user

In [30]:
dg.write_csv('train.csv', dg.train, nb_states=[history_length], nb_actions=[ra_length])

In [8]:
import pandas as pd
import numpy as np
def read_file(data_path):
    ''' Load data from train.csv or test.csv. '''

    data = pd.read_csv (data_path, sep = ';')
    for col in ['state', 'n_state', 'action_reward']:
        data[col] = [np.array ([[np.int (k) for k in ee.split ('&')] for ee in e.split ('|')]) for e in data[col]]
    for col in ['state', 'n_state']:
        data[col] = [np.array ([e[0] for e in l]) for l in data[col]]

    data['action'] = [[e[0] for e in l] for l in data['action_reward']]
    data['reward'] = [tuple (e[1] for e in l) for l in data['action_reward']]
    data.drop (columns = ['action_reward'], inplace = True)

    return data

In [ ]:
data = pd.read_csv ('train.csv', sep = ';')
# for col in ['state', 'n_state', 'action_reward']:
#     data[col] = [np.array ([[np.int (k) for k in ee.split ('&')] for ee in e.split ('|')]) for e in data[col]]
# for col in ['state', 'n_state']:
#     data[col] = [np.array ([e[0] for e in l]) for l in data[col]]




In [ ]:
data['action'] = [[e[0] for e in l] for l in data['action_reward']]
data['reward'] = [tuple (e[1] for e in l) for l in data['action_reward']]
data.drop (columns = ['action_reward'], inplace = True)

In [9]:
data = read_file('train.csv')

In [13]:
data

,state,n_state,action,reward
0,"[82, 845, 923, 477, 471, 699, 134, 502, 613, 4...","[82, 845, 923, 477, 471, 699, 134, 502, 613, 4...","[700, 395, 1114, 728]","(0, 0, 0, 0)"
1,"[286, 190, 344, 245, 736, 558, 879, 535, 898, ...","[286, 190, 344, 245, 736, 558, 879, 535, 898, ...","[1089, 272, 221, 1080]","(0, 1, 1, 1)"
2,"[333, 272, 879, 302, 689, 751, 328, 313, 327, ...","[333, 272, 879, 302, 689, 751, 328, 313, 327, ...","[271, 289, 751, 689]","(1, 1, 1, 1)"
3,"[960, 169, 178, 45, 488, 171, 56, 100, 192, 22...","[960, 169, 178, 45, 488, 171, 56, 100, 192, 22...","[56, 943, 488, 70]","(1, 0, 1, 0)"
4,"[172, 1163, 451, 161, 193, 301, 218, 715, 137,...","[172, 1163, 451, 161, 193, 301, 218, 715, 137,...","[542, 137, 86, 909]","(0, 1, 1, 1)"
...,...,...,...,...
674,"[455, 298, 124, 117, 257, 1007, 1, 993, 475, 2...","[455, 298, 124, 117, 257, 1007, 1, 993, 475, 2...","[1007, 1, 455, 147]","(1, 1, 1, 1)"
675,"[302, 255, 1, 246, 886, 307, 262, 268, 269, 33...","[302, 255, 1, 246, 886, 307, 262, 268, 269, 33...","[237, 255, 1, 150]","(0, 1, 1, 0)"
676,"[181, 258, 746, 721, 127, 173, 33, 174, 117, 1...","[181, 258, 746, 721, 127, 173, 33, 174, 117, 1...","[232, 421, 230, 655]","(0, 1, 0, 1)"
677,"[187, 50, 318, 520, 515, 705, 736, 91, 959, 31...","[187, 50, 318, 520, 515, 705, 736, 91, 959, 31...","[1139, 384, 636, 1140]","(0, 0, 1, 0)"


In [12]:
data['state'][0]

array([  82,  845,  923,  477,  471,  699,  134,  502,  613,  414,  965,
       1147])